In [ ]:
#ler planilha excel,pegar nome, termos banidos, preço minimo , preço maximo 
#abrir navegador, google shopping, pegar todos os resultados que tenham preço dentro da faixa e sejam o os produtos corretos
#o mesmo para o buscape
#enviar um email com uma tabela com os itens e preços encontrados junto com o link de compra

In [237]:
import serpapi
import pandas as pd
import os 
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options


caminho = os.getcwd()
caminho_arquivo = caminho+ r"\buscas.xlsx"
tabela = pd.read_excel(caminho_arquivo) # base de dados
display(tabela)

servico = Service(ChromeDriverManager().install())

options = Options()
options.add_argument('--disable-notifications')



,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 14 128gb,mini watch,3500,4000
1,rtx 3080 ti,zota galax,4000,4500


In [238]:
def pesquisaGoogle(nome,termos_banidos,prec_min,prec_max):
    api_key = 'chavedoserpapi' #chave do serpapi
    client = serpapi.Client(api_key=api_key)
    
    lista_ofertas = []
    resultados = client.search({
        'engine': 'google_shopping',
        'q':nome, # nome do produto
        'gl': 'br', # apenas buscas no Brasil
        'num': 10,
        
    })

    for produto in resultados['shopping_results']:
        preco = produto['extracted_price'] #preco do produto
        nome =  produto['title'].lower() # nome do produto minusculo
        link = produto['link'] # link do produto
        if prec_min<=preco<=prec_max:
            for termo in termos_banidos:
                if termo not in nome:
                    lista_ofertas.append((nome,preco,link))
    
    return lista_ofertas

def pesquisaBuscape(nome,termos_banidos,prec_min,prec_max):
    navegador = webdriver.Chrome(options=options)
    navegador.get("https://www.buscape.com.br/")
    navegador.find_element(By.CLASS_NAME, 'AutoCompleteStyle_input__WAC2Y').send_keys(nome, Keys.ENTER) # pesquisa o produto

    lista_ofertas = []
    lista_prod = navegador.find_elements(By.CLASS_NAME,'ProductCard_ProductCard_Inner__gapsh') #lista com os links dos produtos
    nome_prod = navegador.find_elements(By.CLASS_NAME,'ProductCard_ProductCard_Name__U_mUQ') # lista com os nome dos produtos
    prec_prod = navegador.find_elements(By.CLASS_NAME,'Text_MobileHeadingS__HEz7L') #lista com os preços dos produtos

    for i,nome in enumerate(nome_prod):
        preco = prec_prod[i].text
        preco = float(preco.replace("R$","").replace(" ","").replace(".","").replace(",",".")) #pega o preço e transforma em float
        nome =  nome_prod[i].text 
        nome = nome.lower() # pega o nome do produto e deixa minusculo
        verificacao = True # verificação de termos
        
        for termo in termos_banidos:
            if termo in nome: # verifica se existe ou não o termo banido no nome do produto
                verificacao = False 

        if prec_min<=preco<=prec_max and verificacao==True: # verifica se o produto esta na faixa de valores e continua se não existir termo banido 
            link = lista_prod[i].get_attribute('href')
            lista_ofertas.append((nome,preco,link))

    return lista_ofertas
                    


In [239]:
tabela_ofertas = pd.DataFrame()
for linha in tabela.index:
    nome = tabela.loc[linha,'Nome']
    termos_banidos = tabela.loc[linha,'Termos banidos'].split(" ")
    prec_min = tabela.loc[linha,'Preço mínimo']
    prec_max = tabela.loc[linha,'Preço máximo']
    lista_google = pesquisaGoogle(nome,termos_banidos,prec_min,prec_max)

    if lista_google:
        tabela_google = pd.DataFrame(lista_google, columns=['produto', 'preco', 'link'])
        tabela_ofertas = pd.concat([tabela_ofertas,tabela_google])
    else:
        tabela_google = None

    lista_buscape = pesquisaBuscape(nome,termos_banidos,prec_min,prec_max)
    if lista_buscape:
        tabela_buscape = pd.DataFrame(lista_buscape, columns=['produto', 'preco', 'link'])
        tabela_ofertas = pd.concat([tabela_ofertas,tabela_buscape])

    else:
        tabela_buscape = None


The chromedriver version (127.0.6533.99) detected in PATH at c:\Users\Guilherme\AppData\Local\Programs\Python\Python312\chromedriver.exe might not be compatible with the detected chrome version (128.0.6613.86); currently, chromedriver 128.0.6613.86 is recommended for chrome 128.*, so it is advised to delete the driver in PATH and retry
The chromedriver version (127.0.6533.99) detected in PATH at c:\Users\Guilherme\AppData\Local\Programs\Python\Python312\chromedriver.exe might not be compatible with the detected chrome version (128.0.6613.86); currently, chromedriver 128.0.6613.86 is recommended for chrome 128.*, so it is advised to delete the driver in PATH and retry


In [241]:
tabela_ofertas = tabela_ofertas.reset_index(drop=True)
tabela_ofertas.to_excel("Ofertas.xlsx", index=False) #criando excel